In [1]:
# Basic modules
import numpy as np
import ir_datasets
# The model modules
from libs.lexical_retriever import LexicalRetriever
from libs.vector_retriever import VectorRetriever
from libs.hybrid_retriever import HybridRetriever
from libs.basic_document_storage import BasicStorage

In [2]:
# Preparing the dataset
# Loading the dataset
dataset = ir_datasets.load("cranfield")
# Preparing the docs and queries
prep_funct = lambda x: dict(doc_id=x[0],title=x[1],text=x[2],author=x[3],bib=x[4])
docs = [doc for doc in map(prep_funct,dataset.docs_iter()) if len(doc['text'])]
queries = {query[0]:query[1] for query in dataset.queries_iter()}


In [3]:
# Preparing the Indexer
index = BasicStorage()
# Adding the documents
index.add_documents(docs)

In [4]:
# Preparing the retrievers
vector = VectorRetriever()
lexical = LexicalRetriever()

In [5]:
# Preparing the hybrid retrieval
retriever = HybridRetriever(documents_=index, retrievers=[vector])
retriever.add(docs)

TypeError: cannot pickle '_thread.RLock' object

In [ ]:
# Iter for al queries
queries_info = {}
for query in dataset.qrels_iter():
    query_id = query[0]
    if query[2] >=3:
        exist_info = queries_info.get(query_id,set())
        exist_info.add(query[1])
        queries_info[query_id] = exist_info


In [ ]:
def get_precistion_recall():
    precision = []
    recall = []
    for query_id, relevants_docs in queries_info.items():
        # The text of the query
        if query_id not in queries:
            continue
    
        query = queries[query_id]
        # Get the documents
        retrieved_docs = set([doc['doc_id'] for doc in retriever.search(query,k=25)])
        # relevant and retrieved
        rr = relevants_docs.intersection(retrieved_docs)
        # Calculating the precision
        current_precision = len(rr)/len(retrieved_docs)
        precision.append(current_precision)
        # Calculating the recall
        current_recall = len(rr)/len(relevants_docs)
        recall.append(current_recall)
    
    precision = np.array(precision)
    recall = np.array(recall)
    
    return precision,recall
    

In [ ]:
precision,recall = get_precistion_recall()
print(precision.min(),np.median(precision),precision.mean(),precision.max())
print(recall.min(),np.median(recall),recall.mean(),recall.max())

TypeError: cannot pickle '_thread.RLock' object

In [ ]:
for index,query in queries.items():
    retrieved = set(map(lambda x: int(x['doc_id']),retriever.search(query)))
    valid =set(map(lambda x:int(x),queries_info[index]))
    print(len(retrieved),len(valid.intersection(retrieved)),len(valid))

25 8 21
25 3 17
25 0 2
25 0 6


KeyError: '9'